## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [8]:
import os
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib auto

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
# x axis 9 pionts, y axis 6 points, scan from x axis, one piont by one piont
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# print(objp)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
# print(images)

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)  # corners are 9 x 6 = 54 coordinates

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)   #draw corners in orgnial input image
        cv2.imshow('img',img)
        cv2.waitKey(400)     # if pass parameter 500, wait for 500ms; if leave blank, wait for any key input

cv2.destroyAllWindows()      # when display all images, destroy all windows

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, 
                                            gray.shape[::-1], None, None)

Using matplotlib backend: Qt4Agg


In [9]:
%matplotlib auto
# pick a camera calibration image and undistorted it.
img = cv2.imread('camera_cal/calibration1.jpg')
undist = cv2.undistort(img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image')
ax2.imshow(undist)
ax2.set_title('Undistorted Image')
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

Using matplotlib backend: Qt4Agg


## Apply a distortion correction to raw images.

In [10]:
import matplotlib.image as mpimg
# undistort test image and write to output_images folder
if not os.path.exists('output_images'):
    output_path = 'output_images'
    for img_name in glob.glob('test_images/*.jpg'):
        img = cv2.imread(img_name)
        undist = cv2.undistort(img, mtx, dist, None, mtx)
        # print(output_path+'/'+img_name.split('/')[1])
        cv2.imwrite(output_path+'/'+img_name.split('/')[1],undist)
    
# pick a undistorted output image,display it
img = mpimg.imread('test_images/test1.jpg')
undist = mpimg.imread('output_images/test1.jpg')

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image')
ax2.imshow(undist)
ax2.set_title('Undistorted Image')
plt.subplots_adjust(left=0.02, right=1, top=0.9, bottom=0.)

## Use color transforms, gradients, etc., to create a thresholded binary image.

In [11]:
# define color and x-gradient filter
def image_filter(img, s_thresh=(180, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    # gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_th = (s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])
    h_th = (h_channel > 20) & (h_channel < 40)
    s_binary = np.zeros_like(s_channel)
    s_binary[s_th & h_th] = 1
    #s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return color_binary

undist = mpimg.imread('output_images/test1.jpg')
filtered = image_filter(undist)
# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(img)
ax1.set_title('Undistorted Image', fontsize=40)

ax2.imshow(filtered)
ax2.set_title('Filtered Image', fontsize=40)
plt.subplots_adjust(left=0.02, right=1, top=0.9, bottom=0.)

## Apply a perspective transform to rectify binary image ("birds-eye view").

In [23]:
# pick 4 source points and draw onto undistorted image.
# use mouse to manually pick
undist_temp = cv2.imread('output_images/test1.jpg')
test = np.copy(undist_temp)
m_x = -1
m_y = -1
src_points = []
drawing = False 
click = 0
line_img = np.zeros_like(test)
def on_mouse(event,x,y,flags,param):
    global test, m_x,m_y,drawing, click
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # drawing = not drawing
        click += 1
        drawing = True            
        m_x = x
        m_y = y
        src_points.append([m_x,m_y])
        # print('{} Points:({},{})'.format(click,m_x,m_y))
        if click == 4:
            click = 0
            drawing = False
            pts = np.copy(src_points)
            pts = pts.reshape((-1,1,2))
            print('pts: \n',pts)
            test = np.copy(undist_temp)
            cv2.polylines(test,[pts],True,(0,0,255),4)
            print('source points selected')
    elif event==cv2.EVENT_MOUSEMOVE:    # and flags==cv2.EVENT_FLAG_LBUTTON
        if drawing:
            if click % 2 == 1:
                cv2.line(test,(m_x,m_y),(x,m_y),(255,0,0),2)

# cv2.imshow('undistorted image',test)
cv2.namedWindow('undistorted image')
cv2.setMouseCallback('undistorted image',on_mouse)
key = 0
while (1):
    cv2.imshow('undistorted image',test)
    key = cv2.waitKey(1)
    
    if key == 27:
        break

cv2.destroyAllWindows()
print(src_points)

pts: 
 [[[ 561  484]]

 [[ 763  484]]

 [[1109  694]]

 [[ 286  694]]]
source points selected
[[561, 484], [763, 484], [1109, 694], [286, 694]]


In [26]:
# perspective transform
# img = mpimg.imread('output_images/test1.jpg')
imshape = test.shape
h = imshape[0]
w = imshape[1]
src = [[src_points[0][0]-5,src_points[0][1]-5],
       [src_points[1][0]+5,src_points[1][1]-5],
       [src_points[2][0]+5,src_points[2][1]+5],
       [src_points[3][0]-5,src_points[3][1]+5]]
left_x = src_points[3][0]
right_x = src_points[2][0]
# set dst points as below
dst = [[left_x,0],[right_x,0],[right_x, h],[left_x, h]]
print('dst: ',dst)
# transfer src pionts array
src = np.float32(src)
dst = np.float32(dst)
M = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(test, M, (w,h), flags=cv2.INTER_LINEAR)

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
test = cv2.cvtColor(test,cv2.COLOR_BGR2RGB)
ax1.imshow(test)
ax1.set_title('Undistorted Image', fontsize=40)
warped = cv2.cvtColor(warped,cv2.COLOR_BGR2RGB)
ax2.imshow(warped)
ax2.set_title('Warped Image', fontsize=40)
plt.subplots_adjust(left=0.02, right=1.0, top=0.9, bottom=0.)

dst:  [[286, 0], [1109, 0], [1109, 720], [286, 720]]


## Detect lane pixels and fit to find the lane boundary.